# Train a Deep Koopman model using the Keras framework


## Method based on the following paper (aka, ULE)
Lusch, B.; Kutz, J. N. & Brunton, S. L.
Deep learning for universal linear embeddings of nonlinear dynamics 
Nature communications, Nature Publishing Group, 2018, 9, 1-10

## Original code by dykua 
https://github.com/dykuang/Deep----Koopman

## Modifications by Brendan and Katharina 26/8/21

In [1]:
%killbgscripts

All background processes were killed.


In [2]:
from numba import cuda
cuda.select_device(0)
cuda.close()
from keras import backend as K
K.clear_session()

2022-03-24 23:45:57.409334: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
Using TensorFlow backend.


In [3]:
%matplotlib notebook
%load_ext tensorboard

In [4]:
import tensorflow as tf

# Needed for running custom loss
from tensorflow.python.framework.ops import disable_eager_execution
disable_eager_execution()

gpus = tf.config.experimental.list_physical_devices('GPU')
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Lambda
from tensorflow.keras.optimizers import Adam

Num GPUs Available:  1


2022-03-24 23:45:58.033592: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-03-24 23:45:58.033635: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-03-24 23:45:58.033705: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-24 23:45:58.034106: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 3080 Ti computeCapability: 8.6
coreClock: 1.71GHz coreCount: 80 deviceMemorySize: 11.77GiB deviceMemoryBandwidth: 849.46GiB/s
2022-03-24 23:45:58.034119: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-03-24 23:45:58.035257: I tensorflow/stream_executor/platform

In [5]:
import numpy as np
# Fix the seed, for evaluation
# Remove this to explore generalisability
import random
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

In [6]:
from datetime import datetime
import pickle
import itertools
import os
import warnings

In [7]:
# Load generator libraries
import sys
import h5py
sys.path.append('../fromServer_Keras-HDF5-ImageDataGenerator/')
from h5imagegenerator.generator import HDF5ImageGenerator
from albumentations import Compose,Crop,ToFloat,Normalize

In [8]:
import matplotlib
import matplotlib.pyplot as plt
from IPython.display import HTML
from matplotlib import animation, image
matplotlib.rcParams['animation.embed_limit'] = 2**128

# Load the data

In [9]:
# setting parameters
BATCH_SIZE = 128 # BEST for state, 2 seems to never converge

EPOCHS = 25

# Define the number of historical timesteps used for learning
NUM_HISTORICAL_TRAINING_INPUT_STEPS = 1

# Define the number of timesteps in the latent space (should be same as input and output)
NUM_HISTORICAL_TRAINING_LATENT_STEPS = NUM_HISTORICAL_TRAINING_INPUT_STEPS
# Define the number of timesteps in the output (should be same as input and latent)
NUM_HISTORICAL_TRAINING_OUTPUT_STEPS = NUM_HISTORICAL_TRAINING_INPUT_STEPS 

# Define the number of timesteps used to compute the prediction loss (is 30 in ULE paper)
NUM_HISTORICAL_LOSS_PREDICTION_STEPS = NUM_HISTORICAL_TRAINING_INPUT_STEPS

In [10]:
# Select the number of states in the input
xmax_state = 4 # This assumes the final state is the control
num_control = 1

In [11]:
# Choose dataset
# dataset = "20220308_beamwall"
dataset = "20220309_easy"
directory = '../src/{}'.format(dataset)

In [12]:
# Define dataset parameters (don't need to modify)
X_key = 'state_t'
y_key = 'state_tp1'
classes_key = 'state_tp1'

# Augmentor to set the maximum size of the state, and the number of historical data used for training
augmentor = Compose([
    Crop(x_min=0, y_min=0, x_max=xmax_state, y_max=NUM_HISTORICAL_TRAINING_INPUT_STEPS),
])

In [13]:
# Define generators on the HDF5 files (don't need to modify)
train_generator = HDF5ImageGenerator(
        src = directory+'/train.hdf5',
        X_key=X_key,
        y_key=y_key,
        classes_key = classes_key,
        scaler=False, # Don't normalise data. If you want to do this, use the augmentor above
        labels_encoding=False, # this is fine, as its regression
        batch_size=BATCH_SIZE,
        mode='train',
        augmenter=augmentor,
        augmenter_labels = augmentor, # also crop the outputs, modificated by Brendan
        shuffle=True,
)

valid_generator = HDF5ImageGenerator(
        src=directory+'/valid.hdf5',
        X_key=X_key,
        y_key=y_key,
        classes_key = classes_key,
        scaler=False,
        labels_encoding=False,
        batch_size=BATCH_SIZE,
        mode='train',
        augmenter=augmentor,
        augmenter_labels = augmentor,
        shuffle=True,
)

test_generator = HDF5ImageGenerator(
        src=directory+'/test.hdf5',
        X_key=X_key,
        y_key=y_key,
        classes_key = classes_key,
        scaler=False,
        labels_encoding=False,
        batch_size=BATCH_SIZE,
        mode='train', 
        augmenter=augmentor,
        augmenter_labels = augmentor, 
        shuffle=True,
)

EVALUATION_INTERVAL = len(train_generator._indices) // BATCH_SIZE
VALIDATION_STEPS = len(valid_generator._indices) // BATCH_SIZE

In [14]:
# Print some ranges to explore the data
print(train_generator[0][0][:,:,0].min(),train_generator[0][0][:,:,0].max())
print(train_generator[0][1][:,:,0].min(),train_generator[0][1][:,:,0].max())
print(train_generator[0][0][:,:,1].min(),train_generator[0][0][:,:,1].max())
print(train_generator[0][1][:,:,1].min(),train_generator[0][1][:,:,1].max())
print(train_generator[0][0][:,:,2].min(),train_generator[0][0][:,:,2].max())
print(train_generator[0][1][:,:,2].min(),train_generator[0][1][:,:,2].max())

# View the shape of a batch
print(train_generator[0][0].shape) # Note train_generator[i][0] is the input (xt) data for batch i
print(train_generator[0][1].shape) # Note train_generator[i][1] is the output (xt+1) data for batch i

-1.3013636940312392 1.226839275730759
-1.2613272475469868 1.266432958081564
-1.3668266527313806 0.9714522048917758
-1.4036331130047328 0.9832533627174338
-0.2213204542743128 0.34039393067359924
-0.22173959413637334 0.3391719162464142
(128, 1, 4)
(128, 1, 4)


In [15]:
# Best settings I've found so far for the 2D state data
par = {
       'input steps':NUM_HISTORICAL_TRAINING_INPUT_STEPS,
       'latent steps':NUM_HISTORICAL_TRAINING_LATENT_STEPS,
       'output steps':NUM_HISTORICAL_TRAINING_OUTPUT_STEPS,
       'pred steps': NUM_HISTORICAL_LOSS_PREDICTION_STEPS,
       'batch size': BATCH_SIZE,
       'en dim list': [80,80], # Standard size
       'de dim list': [80,80], # Standard size
#        'en dim list': [64,128,256],  # Large size
#        'de dim list': [256,128,64], # Large size
       'K reg': 10e-14, # Regularization parameter for auxiliary Koopman network 
       'epochs': EPOCHS,
       'num complex': 10,  # hyperparameter for number of conjugate pairs (2 needed for rigid pendulum (no control), unclear how many needed for soft)
       'num real': 0, # hyperparameter for real block (not implemented yet, but also might not be needed)
       'hidden_widths_omega': 170, # the width of the auxiliary network (only one layer)
       'lr': 0.001,
       'alpha_1' : 0.001, # Trade off between prediction and linear error (see loss)
       #'alpha_2' : 10e-9, # this is an infinity norm term from ULE paper, but its not used in the code
       'alpha_3' : 10e-14,
       }
par['num_samples'], par['time steps'], par['input feature dim'] = train_generator[0][0].shape
_, _, par['output feature dim'] = train_generator[0][1].shape
input_shape = (par['input steps'], par['input feature dim'])
output_shape = (par['output steps'], par['output feature dim'])

# Latent dimension
par['latent dim'] = 2*par['num complex'] + par['num real']
latent_shape = (par['latent steps'],)+ (par['latent dim'],)

# Define models

In [16]:
from tensorflow.keras.layers import Input, Reshape
from Architecture_old import _transformer,_pred_K, linear_update
'''
Losses
'''
from keras.losses import mean_squared_error, mean_absolute_error
def S_error(args):
    Y0, Y1 = args
    return tf.reduce_mean(tf.math.squared_difference(Y0,Y1))
def State_loss(yTrue, yPred):
    return tf.reduce_mean(tf.math.squared_difference(encoder(yTrue),KGx))
# The plain old reconstruction loss
def Rec_loss(yTrue, yPred):
    return tf.reduce_mean(tf.math.squared_difference(x_in[:,:,:xmax_state-num_control],decoded_x[:,:,:xmax_state-num_control]))
# The mean reconstruction loss over m steps into the future
def Rec_plus1_loss(yTrue, yPred):
    return tf.reduce_mean(tf.math.squared_difference(yTrue[:,:,:xmax_state-num_control],yPred[:,:,:xmax_state-num_control]))
def customLoss():
    # yTrue is the actual value X_t+1
    # yPred is the predicted reconstruction at X_t+1 (as this is the output of full_model, aka decoded_xp)
    # However, yPred is not actally needed when computing the (custom) loss, instead:
    # 1) L_recon is the MSE between X_t and the prediction of X_t
    # 2) L_pred is the MSE between X_t+m and the prediction of X_t+m, over m timesteps
    # 3) L_lin is the MSE between encoder(X_t+m) and the prediction of encoder(X_t+m), over m timesteps
    def Loss(yTrue, yPred):
        L_recon = Lambda(S_error)([x_in[:,:,:xmax_state-num_control], decoded_x[:,:,:xmax_state-num_control]])
        # According to ULE paper, this should be over 30 steps, instead of 50.
        L_pred = tf.reduce_mean(tf.math.squared_difference(
            # these zeros are same as NUM_HISTORICAL_LOSS_PREDICTION_STEPS
            yTrue[:,0,:xmax_state-num_control],  
            yPred[:,0,:xmax_state-num_control]))
        L_lin = tf.reduce_mean(tf.math.squared_difference(encoder(yTrue),KGx))   
        return  ((par['alpha_1']  * (L_recon+L_pred)) + L_lin)
    return Loss

# Create model

In [17]:
from tensorflow.keras.layers import Input
from Architecture_old import _transformer,_pred_K, linear_update

'''
Input
'''
x_in = Input(input_shape)
'''
Encoder part
'''
Gx = _transformer(x_in, par['latent dim'], par['en dim list'],par['alpha_3'],activation_out='linear')
encoder = Model(x_in, Gx)

'''
linear update in latent space: Predicting via Koopman eigenvalues including control
'''
Koop = _pred_K(Gx, par['num complex'], par['num real'],par['hidden_widths_omega'], par['K reg'],par['alpha_3'],activation_out='linear')
LU = linear_update(output_dim = latent_shape, num_complex = par['num complex'], num_real = par['num real'])
KGx = LU([Gx, Koop])
Knet = Model(x_in, [Koop, KGx]) 

'''
Decoder part
'''
decoder_input = Input(shape = Gx.shape[1:])
decoded = _transformer(decoder_input, par['output feature dim'], par['de dim list'],par['alpha_3'], activation_out='linear')
_decoder = Model(decoder_input, decoded)  
'''
Outputs
'''
decoded_x = _decoder(Gx)
decoded_xp = _decoder(KGx)
'''
Full model
'''
full_model = Model(x_in, decoded_xp)

# Create a  custom Knet to predict on the latent space

In [18]:
'''
Custom Koopman network with input: y_t, and output: y_t+1
In contrast to standard Knet defined above, which takes: input x_t and output: y_t+1 
This allows for multi-step predictions in the latent space
'''
# Disassemble layers
layers = [l for l in Knet.layers]

# Create a new layer thats the latent input (y_t), instead of the state (x_t)
latent_input = Input(shape = Gx.shape[1:])

# starting layer
s_layer = len(encoder.layers)

# # When _pred_K uses only first timesteps of Gx to compute Koop
# Next layer directly connects this latent layer to the Pred_K function, bypassing the encoder
# This is the same as the Pred_K function in Architecture.py
x = layers[s_layer](latent_input)# auxilary dense layer
Koop_l = layers[s_layer+1](x)#Dense

# Define new linear update
KGx_l = LU([latent_input, Koop_l])
custom_Knet = Model(latent_input, [Koop_l, KGx_l])

# Output the models

In [19]:
'''
Models
'''
print(encoder.summary())
print(Knet.summary())
print(custom_Knet.summary())
print(_decoder.summary())
print(full_model.summary(line_length=100))

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1, 4)]            0         
_________________________________________________________________
dense (Dense)                (None, 1, 80)             400       
_________________________________________________________________
activation (Activation)      (None, 1, 80)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 1, 80)             6480      
_________________________________________________________________
activation_1 (Activation)    (None, 1, 80)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 1, 20)             1620      
Total params: 8,500
Trainable params: 8,500
Non-trainable params: 0
___________________________________________________________

# Train the model

In [20]:
'''
training
'''   
# make a folder for the model checkpoints
os.mkdir('models') if not os.path.isdir('models') else None
epoch_ck_dir = 'models/'+datetime.now().strftime("%Y.%m.%d.%H.%M.%S")
os.mkdir(epoch_ck_dir)

# Define tensorboard functions
log_dir = "tensorboard_logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir,write_images=True,histogram_freq=0)

#callbacks
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(epoch_ck_dir+'/weights_epoch_{epoch:08d}',
                                                               save_weights_only=False,save_freq='epoch')

callbacks = [
                    model_checkpoint_callback,
                    tensorboard_callback,
                    tf.keras.callbacks.CSVLogger(epoch_ck_dir+'/history.csv'),
                ]   

In [21]:
#%tensorboard --logdir tensorboard_logs/fit  --port=6007

In [22]:
optimizer=Adam(lr = par['lr'],decay = par['lr']/par['epochs'])

# Compile model
full_model.compile(loss=customLoss(),
                   metrics=[
                       State_loss, Rec_loss,Rec_plus1_loss,
                   ],
                   optimizer=optimizer,
                  )

2022-03-24 23:45:58.909459: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-24 23:45:58.909860: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-03-24 23:45:58.909966: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-24 23:45:58.910360: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 3080 Ti computeCapability: 8.6
coreClock: 1.71GHz coreCount: 80 deviceMemorySize: 11.77GiB deviceMemoryBandwidth: 849.46GiB/s
20

# Either carry on from pretraining, or start new

In [ ]:
# Carry on training

history = full_model.fit(train_generator,epochs = EPOCHS, verbose=1,
                validation_data=valid_generator,
                validation_steps=VALIDATION_STEPS,
                steps_per_epoch=EVALUATION_INTERVAL,
                callbacks = callbacks,
                #workers=16,
                #use_multiprocessing=True,
               )
history = history.history

Epoch 1/25


2022-03-24 23:46:00.047248: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11


 35/787 [>.............................] - ETA: 3s - batch: 17.0000 - size: 128.0000 - loss: 0.0021 - State_loss: 0.0017 - Rec_loss: 0.2068 - Rec_plus1_loss: 0.2071   

2022-03-24 23:46:00.348913: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.11
2022-03-24 23:46:00.353971: I tensorflow/stream_executor/cuda/cuda_blas.cc:1838] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2022-03-24 23:46:00.358264: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2022-03-24 23:46:00.358276: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.
2022-03-24 23:46:00.358286: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1365] Profiler found 1 GPUs
2022-03-24 23:46:00.358391: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcupti.so.11.0'; dlerror: libcupti.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/rl-lab/dkncem_environment/lib/python3.8/site-packages/cv2/../../lib64::/usr/local/cuda-11.0/ta

778/787 [============================>.] - ETA: 0s - batch: 388.5000 - size: 127.9152 - loss: 1.3670e-04 - State_loss: 9.4791e-05 - Rec_loss: 0.0206 - Rec_plus1_loss: 0.0213

/home/rl-lab/dkncem_environment/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '
2022-03-24 23:46:04.206168: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-03-24 23:46:04.206287: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-24 23:46:04.206426: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 3080 Ti computeCapability: 8.6
coreClock: 1.71GHz coreCount: 80 deviceMemorySize: 11.77GiB deviceMemoryBandwidth: 849.46GiB/s
2

INFO:tensorflow:Assets written to: models/2022.03.24.23.45.58/weights_epoch_00000001/assets
787/787 [==============================] - 7s 7ms/step - batch: 393.0000 - size: 127.9161 - loss: 1.3524e-04 - State_loss: 9.3765e-05 - Rec_loss: 0.0204 - Rec_plus1_loss: 0.0210 - val_loss: 8.8132e-06 - val_State_loss: 5.0630e-06 - val_Rec_loss: 0.0018 - val_Rec_plus1_loss: 0.0020
Epoch 2/25
787/787 [==============================] - 6s 8ms/step - batch: 393.0000 - size: 127.9161 - loss: 5.7771e-06 - State_loss: 3.5041e-06 - Rec_loss: 0.0010 - Rec_plus1_loss: 0.0013 - val_loss: 3.3591e-06 - val_State_loss: 2.4196e-06 - val_Rec_loss: 3.5169e-04 - val_Rec_plus1_loss: 5.8775e-04
Epoch 3/25
779/787 [============================>.] - ETA: 0s - batch: 389.0000 - size: 127.9153 - loss: 2.6693e-06 - State_loss: 1.8892e-06 - Rec_loss: 2.6731e-04 - Rec_plus1_loss: 5.1273e-04

In [ ]:
'''
Check losses
'''

# training loss
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
plt.figure()
plt.plot((history['loss']), 'r')
plt.plot((history['State_loss']),'k')
plt.plot((history['Rec_loss']), 'b')
plt.plot((history['Rec_plus1_loss']), 'g')
plt.plot((history['val_loss']), '--r')
plt.plot((history['val_State_loss']),'--k')
plt.plot((history['val_Rec_loss']), '--b')
plt.plot((history['val_Rec_plus1_loss']), '--g')

plt.legend(['loss', 'State_loss', 'Rec_loss','Rec_plus1_loss','val_loss', 'val_State_loss', 'val_Rec_loss','val_Rec_plus1_loss',
           ])
plt.yscale('log')
print(history['State_loss'][::100])

In [ ]:
# Plot the input/output data and the next step prediction
def plot_predictions(dataset_name, num_batches):
    dataset = {
      'train_generator': lambda x: train_generator,
      'valid_generator': lambda x: valid_generator,
      'test_generator': lambda x: test_generator
    }[dataset_name](x)
    
    
    for j in range(num_batches):# plot a number of batches
    
        # Next step prediction for a batch
        xhat_tp1 = full_model.predict(dataset[j][0])
    
        for i in range(BATCH_SIZE):
            plt.plot(np.squeeze([dataset[j][0][i,:,0],dataset[j][1][i,:,0]]),
                     np.squeeze([dataset[j][0][i,:,1],dataset[j][1][i,:,1]]),
                     np.squeeze([dataset[j][0][i,:,2],dataset[j][1][i,:,2]]),
                    '-b', label = 'x_t->x_t+1' if i == 0 and j==0 else "")
            plt.plot(np.squeeze([dataset[j][0][i,:,0],xhat_tp1[i,:,0]]),
                    np.squeeze([dataset[j][0][i,:,1],xhat_tp1[i,:,1]]),
                    np.squeeze([dataset[j][0][i,:,2],xhat_tp1[i,:,2]]),
                    '-r', label = 'x_t->xhat_t+1' if i == 0 and j==0 else "")
    plt.legend()
    plt.xlabel('Theta')
    plt.ylabel('dTheta')
    plt.title(dataset_name)
    
num_batches = 2
fig = plt.figure(figsize=plt.figaspect(0.333))
ax = fig.add_subplot(1, 3, 1, projection='3d')
plot_predictions('train_generator',num_batches)
ax = fig.add_subplot(1, 3, 2, projection='3d')
plot_predictions('valid_generator',num_batches)
ax = fig.add_subplot(1, 3, 3, projection='3d')
plot_predictions('test_generator',num_batches)